In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
classifier = torch.nn.Linear(768,3)
#Output dimmensions is 3 since we have 3 classes (positive, negative and neutral)
#Input dimmension is 768 since that's the dimmension of the last hidden layer of the BERT Model
model = torch.nn.Sequential(model,classifier)
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 2e-5)
#Note Learning Rate must be changed in the real model to optimize results

In [ ]:
path = '/content/drive/MyDrive/LLM Recommender Research project/Coding Resources/Assignment 2/cleaned_reviews.csv'

dataset = pd.read_csv(path, usecols = [0,1])

dataset = dataset.astype(str)

dataset['cleaned_review'] = dataset['cleaned_review'].fillna('')

dataset['Ids'] = dataset['cleaned_review'].apply(lambda x: tokenizer.encode(x, add_special_tokens = True,
  truncation = True, max_length = 512, padding = 'max_length'))

dataset['sentiments'] = dataset['sentiments'].map({'positive':2, 'neutral': 1, 'negative': 0})

In [ ]:
def generate_att_mask(ids):
  return[1 if id != 0 else 0 for id in ids]

dataset['attention_mask'] = dataset.apply(lambda x: generate_att_mask(x['Ids']), axis = 1)

dataset = dataset.drop('cleaned_review', axis = 1)

In [ ]:
dataset = dataset.sample(frac= 0.5, random_state=1).reset_index(drop=True)

In [ ]:
dataset

,sentiments,Ids,attention_mask
0,2,"[101, 2614, 3737, 2003, 2307, 102, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,"[101, 2009, 2573, 2307, 5744, 1998, 4251, 6625...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,2,"[101, 2005, 1996, 3976, 2009, 2307, 2009, 2025...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,2,"[101, 4149, 2023, 8000, 2138, 2043, 2026, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 2204, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,2,"[101, 2049, 2200, 3243, 2017, 2064, 2147, 2302...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,2,"[101, 2204, 3321, 1998, 2614, 3737, 3733, 6556...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,1,"[101, 2498, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,0,"[101, 2043, 2034, 2363, 2023, 5338, 2009, 2004...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,2,"[101, 1045, 2066, 2205, 2172, 2023, 4031, 2307...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
from torch.utils.data import Dataset

class dataframeToDataSet(Dataset):
  def __init__(self, dataframe):
    self.dataframe = dataframe
    self.Ids = dataframe['Ids'].values
    self.attention_mask =dataframe['attention_mask'].values
    self.sentiments = dataframe['sentiments'].values

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index):

    ids = torch.tensor(self.Ids[index], dtype=torch.long)
    attention_mask = torch.tensor(self.attention_mask[index], dtype=torch.long)
    sentiment = torch.tensor(self.sentiments[index], dtype=torch.long)

    return sentiment, ids, attention_mask

In [ ]:
dataset = dataframeToDataSet(dataset)

In [ ]:
def train(model, loss_function, optimizer, epochs, batch_size):
  model.train()
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  for epoch in range(epochs):
    for batch in dataloader:

      optimizer.zero_grad()

      sentiment, ids, attention_mask = batch

      BERT_model = model[0](input_ids=ids, attention_mask=attention_mask)
      predicted_sentiment = model[1](BERT_model.last_hidden_state[:, 0, :])

      predicted_sentiment = torch.sigmoid(predicted_sentiment)

      #write code that select the class with the highest probability (Class is represented by index)

      loss = loss_function(predicted_sentiment, sentiment)
      loss.backward()
      optimizer.step()
      print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

In [ ]:
Epochs = 1
Batch_Size = 4
#Change these three parameters to optimize the results in the case of the real model
train(model, loss_func, optimizer, Epochs, Batch_Size)

Epoch: 1, Loss: 1.1179159879684448
Epoch: 1, Loss: 1.0917164087295532
Epoch: 1, Loss: 1.06669282913208
Epoch: 1, Loss: 1.0461581945419312
Epoch: 1, Loss: 1.0294947624206543


In [ ]:
#Tester Code
def test_model(model, review):
  id = tokenizer.encode(review, add_special_tokens = True,
                         truncation = True, max_length = 512, padding = 'max_length')
  attention_mask = [1 if i != 0 else 0 for i in id]

  id = torch.tensor([id])  # Add a batch dimension
  attention_mask = torch.tensor([attention_mask])

  bert_output = model[0](input_ids = id, attention_mask = attention_mask)
  sentiment = model[1](bert_output.last_hidden_state[:, 0, :])

  sentiment = torch.sigmoid(sentiment)
  sentiment = torch.argmax(sentiment, dim=1)
  map = {2: 'positive', 1: 'neutral', 0: 'negative'}
  sentiment = map[sentiment.item()]
  return sentiment

In [ ]:
#Tester Code
review = "Product is amazing!!!"

sentiment = test_model(model, review)

print(sentiment)

positive
